# VotecounterTest
A script for setting up and executing performance tests on votecounters.

## Dependencies

In [1]:
import json
import time
import numpy as np
from VoteCount import VoteCount
import VoteCounter

ModuleNotFoundError: No module named 'enchant'

In [3]:
archive = '../data/archive.txt'
votecounter = VoteCounter

NameError: name 'VoteCounter' is not defined

## Helper Function(s)

In [4]:
def _relevantGameInfo(game):
    link = game[:game.find('\n')]
    number = (link[link.find('&t=')+3:] if link.count('&')==1 
                      else link[link.find('&t=')+3:link.rfind('&')])
    slots, players, correct = [], [], None
    for line in game[game.find('\nPlayers\n')+9:].split('\n'):
        line = line.split(', ')
        players += line[0].split(' replaced ')
        slots.append(line[0].split(' replaced ')) 
        if (line[2].lower().count('lynched') > 0 and
            line[2].lower().count('day 1') > 0):
            correct = slots[-1]
    return slots, players, correct, number

## The VotecounterTest Class

In [5]:
class VotecounterTest:
    
    def __init__(self, archive='archive.txt', posts='posts.jl'):
        # open game archive, separate by game
        with open(archive) as f:
            self.archive = f.read().split('\n\n\n')
        
    def test(self, votecounter):
        # process votes in each game's posts until a lynch found
        # then store information about votecounter's performance
        results, success, t0 = {}, 0, time.time()
        for game in self.archive[:1]:
            slots, players, correct, number = _relevantGameInfo(game)
            print( slots, players, correct, number)
            votecount = VoteCount(slots, meta={'correct': correct})
            votecounter = votecounter.VoteExtracter(players=players)
            
            # collect gameposts associated with game number
            with open('{}.jsonl'.format(number)) as f:
                gameposts = sorted(json.loads(f.read()),
                                   key=lambda x: (int(x['thread']), int(x['number'])))
            
            for post in gameposts:
                # done if voters have made a choice already
                if votecount.choice:
                    break
                
                # ignore posts not made by players
                if players.count(post['user']) == 0:
                    continue
                
                # update votecount for each vote found by votecounter
                for voted in votecounter.fromPost(post):
                    votecount.update(post['user'], voted)
                    if votecount.choice:
                        success += votecount.choice == correct
                        break
            results[number] = votecount
        
        return results, success/float(len(self.archive)), time.time()-t0

## Test

In [17]:
results, successrate, time = VotecounterTest().test(VoteCounter)

MemoryError: 

In [7]:
with open('../data/archive.txt') as f:
    archive = f.read().split('\n\n\n')